In [1]:
from SALib.sample import saltelli
from tqdm.auto import tqdm
import numpy as np
import pickle
import time
import lzma
import sys
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})

plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [2]:
# problem definition
PROBLEM = {
    "num_vars" : 5,
    "names"    : ["theta",
                  "gain_right",
                  "saving_prop",
                  "prob_left",
                  "alpha"],
    "bounds"   : [[0.05, 0.95],
                  [1.70, 8.00],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [2.00, 12.0]]
}

# generate Saltelli samples
NUM_SAMPLES = 1024
X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)

<ipython-input-2-1647d18ac9f5>:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)


In [3]:
os.chdir("../")
import cpt_optimisation

In [4]:
folders = [x for x in os.listdir("DATA/nodebt/") if "DS_" not in x]
folders = sorted(folders, key=lambda x: int(x.split("_")[-1]))
num_agents = 1225

In [30]:
regime = np.zeros((20, 7168))
start_t = time.time()

for seed_folder in folders:
    print(f"{time.time()-start_t:.2f} | {seed_folder}")
    seed = int(seed_folder.split("_")[-1])
    files = os.listdir(f"DATA/nodebt/{seed_folder}")
    files = sorted(files, key=lambda x: int(x.split("_")[1]))
    
    for f in tqdm(files):
        
        param_idx = int(f.split("_")[1]) - 1
        data = pickle.load(lzma.open(f"DATA/nodebt/{seed_folder}/{f}"))
        data_W = data["W"]
        
        alpha_val = data["params"][4]
        with open(f"sda_graphs/{seed}_{alpha_val}.pickle", "rb") as sda_file:
            _, _, augmented_comms, _ = pickle.load(sda_file)

        # determine regime
        richer = 0
        for _,agent_indices in augmented_comms.items():
            if np.sum(data_W[-1][agent_indices]) > np.sum(data_W[0][agent_indices]):
                richer += 1
        prop_richer = richer / len(augmented_comms)
        if prop_richer == 0:
            regime[seed][param_idx] = 0
        elif prop_richer < 1:
            regime[seed][param_idx] = 1
        else:
            regime[seed][param_idx] = 2

0.00 | new_model_runs_paper_0


  0%|          | 0/7168 [00:00<?, ?it/s]

977.65 | new_model_runs_paper_1


  0%|          | 0/7168 [00:00<?, ?it/s]

1918.66 | new_model_runs_paper_2


...

  0%|          | 0/7168 [00:00<?, ?it/s]

15315.68 | new_model_runs_paper_18


  0%|          | 0/7168 [00:00<?, ?it/s]

16153.81 | new_model_runs_paper_19


  0%|          | 0/7168 [00:00<?, ?it/s]

In [31]:
with open("community_regimes.pickle", "wb") as f:
    pickle.dump(regime, f)

In [37]:
np.sum(regime.flatten()==0), np.sum(regime.flatten()==1), np.sum(regime.flatten()==2)

(17766, 88111, 37483)

In [38]:
17766+88111+37483

143360

In [40]:
for i in range(3):
    print(round(np.sum(regime.flatten()==i)/143360, 3))

0.124
0.615
0.261


In [41]:
12+62+26

100